### Import Packages

In [1]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
from nltk.corpus import stopwords
import string
from nltk import word_tokenize,ne_chunk,pos_tag,wordpunct_tokenize
import rake_nltk
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer,PorterStemmer
from itertools import chain, groupby, product
from collections import defaultdict, Counter
import networkx as nx
from networkx.algorithms import community
from gensim.models import Word2Vec,KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.ldamodel import LdaModel
import gensim
import statsmodels.api as sm
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from scipy.stats import ttest_ind
from scipy.stats import normaltest
% matplotlib notebook

//anaconda/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Load Data

In [3]:
all_data_store_path = '/Users/tylio/Google Drive/2018 spring/686/Wiley Sponsered Project/Data'
current_data_folder_name = '(40000+)_by_year'
current_data_store_path = os.path.join(all_data_store_path,current_data_folder_name)
data = pd.read_csv(os.path.join(current_data_store_path,'wholeFile_with_wholeTopic_v2_2.csv'))

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,3,4,6,7,10,40,48,50,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
topic_map = pd.read_csv(os.path.join(current_data_store_path,'topic_map_v2_2.csv'))

### Classify the General Trend of the Topics as Shrinking or Growing

In [5]:
# def get_moving_average_array(series,lag):
#     help_df = pd.DataFrame(series)
#     for i in range(lag-1):
#         help_df.loc[:,i+1] = series.shift(-(i+1))
#     result = help_df.mean(axis = 1,skipna = False).dropna()
#     return result
        
# def get_X_array_MA(lag):
#     array = list(range(1995,2017-lag+2))
#     median = np.median(array)
#     result = (np.array(array)-median)*0.01
#     return result

def get_X_array_normal(start_year,end_year):
    array = list(range(start_year,end_year))
    median = np.median(array)
    result = (np.array(array)-median)*0.01
    return result

def get_Xy_for_ols(start_year,end_year,series,lag = 2,MA = False):
    if MA:   
        X = get_X_array(lag)
        y = get_moving_average_array(series,lag)
    else:
        X = get_X_array_normal(start_year,end_year)
        y = series
    return X.reshape(len(X),1),y
        
#     slide = array[:lag]
#     result.append(np.mean(slide))
#     for i in range(len(array)-lag+1):
        
#         result.append(np.mean(slide))
        

In [6]:
for_reach_maxmin_recent = topic_map[[str(i) for i in range(1995,2018)]]

def reach_min_recent(row):
    vally = int(row.idxmin())
    if vally>=2013:
        return 1

def reach_max_recent(row):
    peak = int(row.idxmax())
    if peak>=2013:
        return 1
    
def get_ES_index(row):
    peak = row['1995']
    vally = row['2017']
    return peak-vally

In [7]:
#topic_map = topic_map[[i for i in topic_map.columns if not i.startswith('19')]]
start_year = 2000
#if start_year>=2000:
 #   topic_map = topic_map[[i for i in topic_map.columns if not i.startswith('19')]]
topic_map = topic_map[topic_map[[str(i) for i in range(start_year,2018)]].max(axis = 1)>0.01]
ignore = ['new','novel','biomaterials standard report','applied biomaterials standard','via','human','fatigue','blood','use','method']
topic_map = topic_map[~topic_map['topic'].isin(ignore)].reset_index(drop = True)
topic_map.loc[:,'number_of_increase'] = (topic_map.loc[:,[str(i)+'(gap)' for i in range(start_year,2018)]]>0).sum(axis = 1)
topic_map.loc[:,'mostly_increase'] = (topic_map.loc[:,[str(i)+'(gap)' for i in range(start_year,2018)]]>0).sum(axis = 1)>int((2018-start_year)/2)


In [8]:
seed = topic_map[[str(i) for i in range(start_year,2018)]]

result = []
for row in seed.index:
    X,y = get_Xy_for_ols(start_year,2018,seed.loc[row],lag = 0,MA = False)
    line_fit = LinearRegression()
    line_fit.fit(X,y)
    coef = line_fit.coef_
    result.append(coef)
topic_map.loc[:,'linear_coef'] = result

//anaconda/lib/python3.5/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [9]:
for_reach_maxmin_recent = topic_map[[str(i) for i in range(start_year,2018)]]
topic_map.loc[:,'reach_max_recent'] = for_reach_maxmin_recent.apply(lambda x:reach_max_recent(x),axis = 1)
topic_map.loc[:,'reach_min_recent'] = for_reach_maxmin_recent.apply(lambda x:reach_min_recent(x),axis = 1)

In [10]:
topic_map.loc[(topic_map['mostly_increase']==True) & (topic_map['linear_coef']>0) & (topic_map['reach_max_recent']==1),'emerging']=1

In [11]:
topic_map.loc[(topic_map['mostly_increase']==False) & (topic_map['linear_coef']<0) & (topic_map['reach_min_recent']==1),'shrinking'] = 1

In [12]:
emerging_topics = topic_map[topic_map['emerging']==1]
shrinking_topics = topic_map[topic_map['shrinking']==1]

In [13]:
emerging_topic_list = emerging_topics['topic'].values
shrinking_topic_list = shrinking_topics['topic'].values

In [35]:
topic_map.loc[(topic_map['emerging'].isnull())&(topic_map['shrinking'].isnull()),'neutral'] = 1

In [40]:
topic_map.head()

,topic,doc_index,1995,1996,1997,1998,1999,2000,2001,2002,...,linear_coef,reach_max_recent,reach_min_recent,emerging,shrinking,emerging_period_start,emerging_period_end,shrinking_period_start,shrinking_period_end,neutral
0,polyester,105;125;274;326;392;549;560;582;652;692;1010;1...,0.003333,0.008403,0.000000,0.002193,0.008016,0.007313,0.010000,0.010974,...,-0.024182,NaN,1.0,NaN,1.0,NaN,NaN,2005.0,2009.0,NaN
1,investigation,302;432;656;659;777;802;827;1038;1148;1626;174...,0.006667,0.002801,0.005249,0.006579,0.004008,0.005484,0.003333,0.009602,...,0.001517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,biofilm,147;159;231;272;611;678;712;837;896;1035;1144;...,0.003333,0.005602,0.005249,0.004386,0.004008,0.005484,0.006667,0.000000,...,0.018618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,self-assembly,35;163;292;460;480;523;541;545;594;597;706;719...,0.006667,0.000000,0.000000,0.013158,0.002004,0.003656,0.008333,0.005487,...,0.104989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,gel,1;149;501;596;702;727;999;1033;1066;1106;1116;...,0.006667,0.008403,0.007874,0.010965,0.008016,0.012797,0.003333,0.010974,...,0.023970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


### Detect the Shrinking Period and Growing Period for Each of the Topic

In [16]:
def movingFit(index,start_year = 2000,end_year = 2018,window = 5):
    year_range = list(range(start_year,end_year))
    X = get_X_array_normal(start_year,start_year+window)
    X = X.reshape(len(X),1)
    left = start_year
    right = start_year+window
    coefs = []
    while(right<=end_year):
        y = topic_map.loc[index,[str(i) for i in range(left,right)]]
        line_fit = LinearRegression()
        line_fit.fit(X,y)
        coef = line_fit.coef_
        coefs.append(coef[0])
        left+=1
        right+=1
    emerging_period = (year_range[np.argmax(coefs)],year_range[np.argmax(coefs)]+4)
    shrinking_period = (year_range[np.argmin(coefs)],year_range[np.argmin(coefs)]+4)
#     if max(coefs)<0:
#         emerging_period = (np.NaN,np.NaN)
#     else:
#         emerging_period = (year_range[np.argmax(coefs)],year_range[np.argmax(coefs)]+4)
#     if min(coefs)>0:
#         shrinking_period = (np.NaN,np.NaN)
#     else:
#         shrinking_period = (year_range[np.argmin(coefs)],year_range[np.argmin(coefs)]+4)
    return emerging_period,shrinking_period

In [17]:

for i in emerging_topics.index:
    temp = movingFit(i)[0]
    #emerging_period.append(temp)
    topic_map.loc[i,'emerging_period_start'] = temp[0]
    topic_map.loc[i,'emerging_period_end'] = temp[1]

for i in shrinking_topics.index:
    temp = movingFit(i)[1]
    #emerging_period.append(temp)
    topic_map.loc[i,'shrinking_period_start'] = temp[0]
    topic_map.loc[i,'shrinking_period_end'] = temp[1]

In [75]:
topic_map = pd.read_csv(os.path.join(current_data_store_path,'topic_map_v2_2.csv'))

In [18]:
for t in emerging_topic_list:
    data.loc[data[t]==1,'emerging'] = 1
for t in shrinking_topic_list:
    data.loc[data[t]==1,'shrinking'] = 1
data.loc[(data['emerging']==1)&(data['shrinking']==1),'emergingAndShrinking'] = 1

In [41]:
topic_map.to_csv(os.path.join(current_data_store_path,'topic_map_trend_v2_2.csv'),index=False)

In [20]:
data.to_csv(os.path.join(current_data_store_path,'wholeFile_with_wholeTopic_v2_2.csv'),index=False)